In [3]:
import pandas as pd
import numpy as np
import tia.bbg.datamgr as dm
mgr=dm.BbgDataManager()

#use 'Book1.xlsx' as the example
print("What file are you selecting?")
files=input()
df = pd.read_excel(files)
mgr.sid_result_mode = 'frame'

c=0
empty=[]
for x in df['CUSIP']:
    empty.append('/cusip/'+str(df.at[c,'CUSIP']))
    c=c+1
    next
liss=[]
for b in empty:
    store=mgr[b]
    first=store['PX_LAST','PAR_AMT','CPN','CPN_FREQ','DAYS_TO_NEXT_COUPON','DAYS_TO_MTY']
    liss.append(first)
g=2

dataset=liss[0]
dataset=liss[0].append(liss[1])
for i in range(2,len(liss)):
    dataset=dataset.append(liss[g])
    g=g+1

def Build_info(x):    
    dataset['COUPON_PAYMENT']=(dataset['CPN']*dataset['PAR_AMT'])/100
    dataset['DTM_DNC']=dataset['DAYS_TO_MTY']-dataset['DAYS_TO_NEXT_COUPON']
    dataset['YR_FREQ']=365/dataset['CPN_FREQ']
    dataset['#_OF_CPN']=np.rint(dataset['DTM_DNC']/dataset['YR_FREQ'])
    return;

def Duration_calc(x):
        
    for index, row in dataset.iterrows():
        #other
        ation=((row['DAYS_TO_NEXT_COUPON']*row['COUPON_PAYMENT'])+(((row['COUPON_PAYMENT']+row['PAR_AMT'])*row['DAYS_TO_MTY']))/row['PX_LAST'])
    
        #final
        dur=(((row['COUPON_PAYMENT']+row['PAR_AMT'])*row['DAYS_TO_MTY'])/row['PX_LAST'])
        if row['DTM_DNC']>row['YR_FREQ']:
            #normal
            store_val=0
            for x in range(row['#_OF_CPN'].astype(np.int64)):
                sols=0
                sols=row['COUPON_PAYMENT']*(row['DAYS_TO_NEXT_COUPON']+(row['YR_FREQ']*x))
                store_val=store_val+sols
            dataset.at[index,'DURATION']="{:10.0f}".format(ation+store_val)
        if row['DTM_DNC']<=row['YR_FREQ']:
            if row['DAYS_TO_NEXT_COUPON']==row['DAYS_TO_MTY']:
                #final
                dataset.at[index,'DURATION']="{:10.0f}".format(dur)
            if row['DAYS_TO_MTY']>row['YR_FREQ']:
                #other
                dataset.at[index,'DURATION']="{:10.0f}".format(ation)
    return;       



What file are you selecting?
'Book1.xlsx'


In [4]:
dataset

,PX_LAST,PAR_AMT,CPN,CPN_FREQ,DAYS_TO_NEXT_COUPON,DAYS_TO_MTY
/cusip/9128286B1,101.468750,100.0,2.625,2.0,94.0,3566.0
/cusip/88032XAN4,100.532000,1000.0,3.975,2.0,150.0,3620.0
/cusip/912828W71,99.406250,100.0,2.125,2.0,140.0,1784.0
/cusip/912810SF6,102.515625,100.0,3.000,2.0,94.0,10871.0
/cusip/912810SE9,110.031250,100.0,3.375,2.0,2.0,10779.0


In [5]:
Build_info(dataset)

In [6]:
dataset

,PX_LAST,PAR_AMT,CPN,CPN_FREQ,DAYS_TO_NEXT_COUPON,DAYS_TO_MTY,COUPON_PAYMENT,DTM_DNC,YR_FREQ,#_OF_CPN
/cusip/9128286B1,101.468750,100.0,2.625,2.0,94.0,3566.0,2.625,3472.0,182.5,19.0
/cusip/88032XAN4,100.532000,1000.0,3.975,2.0,150.0,3620.0,39.750,3470.0,182.5,19.0
/cusip/912828W71,99.406250,100.0,2.125,2.0,140.0,1784.0,2.125,1644.0,182.5,9.0
/cusip/912810SF6,102.515625,100.0,3.000,2.0,94.0,10871.0,3.000,10777.0,182.5,59.0
/cusip/912810SE9,110.031250,100.0,3.375,2.0,2.0,10779.0,3.375,10777.0,182.5,59.0


In [7]:
Duration_calc(dataset)

In [13]:
dataset.to_excel('output1.xlsx')